In [1]:
#!pip install gradio

In [2]:
from huggingface_hub import InferenceClient
import gradio as gr

In [3]:
client = InferenceClient(
    "mistralai/Mistral-7B-Instruct-v0.1"
)

In [4]:
#!pip install modin[all]

In [5]:
import modin.pandas as pd

In [6]:
#!pip install intel-extension-for-pytorch

ERROR: Could not find a version that satisfies the requirement intel-extension-for-pytorch (from versions: none)
ERROR: No matching distribution found for intel-extension-for-pytorch


In [7]:
import sys
import os
#!pip install datasets tensorboardX

In [8]:
#!pip install torch==2.2.0

In [9]:
#!pip install PyPDF2

In [10]:
#!pip install llama_index

In [11]:
'''from pathlib import Path
from llama_index import download_loader

PDFReader = download_loader("PDFReader")'''

ImportError: cannot import name 'download_loader' from 'llama_index' (unknown location)

In [13]:
import logging
import sys
import torch
from pprint import pprint
#from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
#from llama_index.llms import HuggingFaceLLM
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
# from llama_index import LangchainEmbedding
#from llama_index.prompts.prompts import SimpleInputPrompt

In [14]:
import modin.pandas as pd
#this method takes two pages at a time in a sliding window fashion and generates questions and answers for them
def create_qa_set_for_documents(documents_list):
    all_pairs = []
    for i in range(0, len(documents_list)-1):
        print("Document pair no. - "+str(i))
        text = documents_list[i].text
        if i < len(documents_list)-1 :
            text = text + documents_list[i+1].text
        documents_pair = [documents_list[i].copy()]
        documents_pair[0].text = text
        all_pairs.extend(create_qa_set(text, documents_pair))
        print("Total Pairs so far: "+str(len(all_pairs)))
    return all_pairs

In [15]:
#question creation prompts
q_system_prompt = "You are an expert user extracting questions from a text. You will be passed a piece of text which would be the problem of patients, write a list of questions that can be answered based *solely* on the given text."
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

#answer prompt
a_system_prompt = "You are a ayurvedic remedies/herbs/medicine recommender. Extract the exact data from given document and give them as responsee in a pointwise manner. If no information found please reply 'no informa tion available'"

In [20]:
#loader = PDFReader()
data = []
try:
loader.load_data(file=Path(r'C:\Users\hp\Downloads\Ayurvedic Dataset.pdf'))
except:
    print("error")

NameError: name 'loader' is not defined

In [ ]:
def getLLMforPrompts(system_prompt, query_wrapper_prompt) :
    return HuggingFaceLLM(
        context_window=context_window,
        max_new_tokens=256,
        generate_kwargs={"temperature":temperature,"top_p":0.5, "do_sample": False},
        system_prompt= system_prompt,
        query_wrapper_prompt = query_wrapper_prompt,
        tokenizer=tokenizer,
        model=model_llm,
        device_map='auto',
        model_kwargs={"temperature":0.1, "top_p":0.5,"use_auth_token": True}
    )

In [ ]:
def getAllQuestions(response_text):
    j=0
    lines = response_text.split("\n")
    list_questions = []
    for i in range(0, len(lines)):
        if "?" in lines[i] and not lines[i][0].isalpha():
            for j in range(0, len(lines[i])):
                if lines[i][j].isalpha():
                    break
            list_questions.append(lines[i][j:].replace("  "," ").strip())
    if len(list_questions) == 0:
        for i in range(0, len(lines)):
            if "?" in lines[i]:
                list_questions.append(lines[i][j:].replace("  "," ").strip())
    list_questions = list(set(list_questions))
    return list_questions

In [ ]:
def getQueryEngine(a_llm, docs):
    service_context = ServiceContext.from_defaults(
        chunk_size=1024,
        llm=a_llm,
        embed_model=embed_model
    )
    index = VectorStoreIndex.from_documents(docs, service_context=service_context)
    return index.as_query_engine()